In [24]:
from requests import get 

response = get(f"https://forecast.weather.gov/MapClick.php?lat=41&lon=-87&FcstType=digitalDWML")
for x in response.text.split("\n"):
    print(x)

<?xml version="1.0" encoding="ISO-8859-1"?>
<dwml version="1.0" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://graphical.weather.gov/xml/DWMLgen/schema/DWML.xsd">
  <head>
    <product concise-name="tabular-digital" operational-mode="developmental" srsName="WGS 1984">
      <creation-date refresh-frequency='PT1H'>2024-01-28T17:57:16-06:00</creation-date>
    </product>
    <source>
      <production-center>Chicago, IL</production-center>
      <credit>https://www.weather.gov/lot</credit>
      <more-information>https://www.nws.noaa.gov/forecasts/xml/</more-information>
    </source>
  </head>
  <data>

  <location>
    <location-key>point1</location-key>
    <point latitude="41" longitude="-87.01"/>
    <area-description>6 Miles W Francesville IN</area-description>
    <height datum="mean sea level" height-units="feet">689</height>
  </location>
<moreWeatherInformation applicable-location="point1"

In [39]:
def getXMLElements(xmlInput, tagName, attributeNames=[], attributeValues=[]):

    # - Inputs -
    # xmlInput - list or tuple of xml rows split on "\n" 
    # 
    # tagName - string tagename that this function will look for
    #                   tagName example
    # <temperature type="dew point" time-layout="k-p1h-n1-0"><value>-7</value><value>-7</value><value>-6</value><value>-7...
    # temperature would be the tagname you would want to provide if you want this function to return this row
    #
    # attributeNames - list of strings that are attributes you would like to match, the order must correspond with the order of the attributeValues, optional
    #                    attributeNames example
    # attributeNames = ["type","time-layout"]
    # type and time-layout would be matched with the first and second entires in the attributeValues list
    #
    # attributeValues - a list of strings that corresponds to the order of attributes in attributeNames, this is optional
    #                     attributeNames example
    # attributeNames = ["dew point", "k-p1h-n1-0"]
    #
    # This function will return a tuple of strings for all the elements that match in the xmlFileName with the tagname and provided attribute information

    ###################

    #
    # First we will narrow down the file based on tagname alone and create a list of elements of interest
    #

    # the below pattern will pull out all the possible attributes assosiated with a specific tag and append them to the list attributeNames

    import re
    import gc

    patt = re.compile(f"<{tagName}(.*?)</{tagName}>")
    elementsFiltered = []

    while xmlInput:
        xmlLine = xmlInput.pop()
        n = 0
        while n < len(xmlLine):
            regSearch = patt.search(xmlLine[n:])
            if regSearch != None:
                n += regSearch.end()
                elementsFiltered.append(regSearch.group(0))
            else:
                break
  
    del regSearch, patt, n, xmlLine 

    elementsFiltered2 = []

    if attributeNames:
        while elementsFiltered:
            element = elementsFiltered.pop()
            correctAttributes = False
            
            for b in range(len(attributeNames)):

                patt = re.compile(f"{attributeNames[b]}=\"(.*?)\"")
                searchResult = patt.search(element)

                if searchResult == None:
                    correctAttributes = False
                    break
                elif searchResult.group(1) != attributeValues[b]:
                    correctAttributes = False
                    break
                elif searchResult.group(1) == attributeValues[b]:
                    correctAttributes = True

            if correctAttributes:
                elementsFiltered2.append(element)

            del searchResult, patt, correctAttributes, b, element

        return (elementsFiltered2)

    else:
        return (elementsFiltered)
    

xmlInput = response.text.split("\n")
tagName = "wind-speed"
attributeNames = []
attributeValues = ["sustained"]
rep = getXMLElements(xmlInput, tagName, attributeNames, attributeValues)

rep

['<wind-speed type="gust" time-layout="k-p1h-n1-0"><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value xsi:nil="true"/><value>14</value><value>15</value><value>15</value><value>15</value><value xsi:nil="true"/><value>14</value><value>16</value><value>18</value><value>21</value><value>21</value><value>21</value><value>21</value><value>20</value><value>20</value><value>18</value><value>17</value><value>16</value><value>15</value><value>15</value><value>15</value><value>15</value><value>15</value><value>15</value><value>15</value><value>15</value><value>15</value><value>15</value><value>14</value><value>14</value><value>13</value><value xsi:nil="true"/><value xsi:nil="tru

In [35]:
len(rep)

1

'<cap:expires>2024-01-29T09:00:00-06:00</cap:expires>'